# Generate Prompt Yelp Advisor

In [39]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Part
import vertexai.preview.generative_models as generative_models

In [40]:
vertexai.init(project="adsp-capstone-property-pilot", location="us-central1")

In [41]:
model = GenerativeModel("gemini-1.0-pro")
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

### Test

In [42]:
prompt = "Hello."
print(get_chat_response(chat, prompt))

Hello! How can I help you today?


### Yelp Advisor

In [43]:
import http.client
import json
import urllib.parse
from typing import Dict, Any
import folium
import os

In [44]:
os.chdir('/home/jupyter/property_pilot/prompts')
# yelp_advisor_instructions
file_path = 'yelp_advisor_instructions.txt'
try:
    with open(file_path, 'r') as file:
        yelp_advisor_instruction = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")

In [45]:
os.chdir('/home/jupyter/property_pilot/prompts')
# yelp_categories
file_path = 'yelp_categories.txt'
try:
    with open(file_path, 'r') as file:
        yelp_categories = file.read().splitlines()
        # Format the list into a single string separated by commas for the model
        categories_string = ','.join(yelp_categories)
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")

In [52]:
# Function to generate the API filter parameters based on user query
def generate_prompt_yelp_advisor(yelp_advisor_instruction, user_query, categories_string):
    prompt = yelp_advisor_instruction.replace("{USER_QUERY}", user_query)
    prompt = prompt.replace("{CATEGORIES_STRING}", categories_string)
    return prompt

In [53]:
# Function to ensure valid JSON
def extract_json_to_dict(text):
    start = text.find('{')
    end = text.rfind('}') + 1

    if start != -1 and end != -1:
        json_str = text[start:end]
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return "Invalid JSON format"
    else:
        return "No valid JSON object found"

### Test

In [54]:
type(yelp_advisor_instruction)

str

In [55]:
type(categories_string)

str

In [60]:
user_query = "What are the best Thai restaurants in Chicago?"

yelp_prompt = generate_prompt_yelp_advisor(yelp_advisor_instruction, user_query, categories_string)
#print(yelp_prompt)

yelp_filter_str = get_chat_response(chat, yelp_prompt)
print(yelp_filter_str)

yelp_filter = extract_json_to_dict(yelp_filter_str)
print(yelp_filter)

```json
{
"term": "thai",
"location": "Chicago, IL",
"sort_by": "rating",
"limit": 5
}
```
{'term': 'thai', 'location': 'Chicago, IL', 'sort_by': 'rating', 'limit': 5}


In [58]:
user_query = "What are the best budget-friendly restaurant options in South Loop?"

yelp_prompt = generate_prompt_yelp_advisor(yelp_advisor_instruction, user_query, categories_string)

yelp_filter_str = get_chat_response(chat, yelp_prompt)
print(yelp_filter_str)

yelp_filter = extract_json_to_dict(yelp_filter_str)
print(yelp_filter)

```
{
"location": "South Loop, Chicago, IL",
"term": "restaurants",
"price": [1, 2],
"sort_by": "rating",
"limit": 5
}
```
{'location': 'South Loop, Chicago, IL', 'term': 'restaurants', 'price': [1, 2], 'sort_by': 'rating', 'limit': 5}


In [59]:
user_query = "What are the top five restaurants in West Loop?"

yelp_prompt = generate_prompt_yelp_advisor(yelp_advisor_instruction, user_query, categories_string)

yelp_filter_str = get_chat_response(chat, yelp_prompt)
print(yelp_filter_str)

yelp_filter = extract_json_to_dict(yelp_filter_str)
print(yelp_filter)

```json
{
"location":"West Loop, IL",
"term":"restaurants",
"sort_by":"rating",
"limit":5
}
```
{'location': 'West Loop, IL', 'term': 'restaurants', 'sort_by': 'rating', 'limit': 5}
